# Linear regression with scikit-learn

Importing the modules

In [46]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.dates import DateFormatter

import numpy as np
from sklearn import linear_model

from cassandra.cluster import Cluster
from cassandra.metadata import KeyspaceMetadata

Connecting to SensorHUB

In [47]:
cluster = Cluster(["172.20.16.137"], port=30241)
keyspace_name = "sensorhub"
session = cluster.connect(keyspace_name)

Getting data

In [48]:
query = "SELECT sensor, datum, temperatureat2meter FROM erti_new WHERE datum >= '2017-01-01 00:00:00' AND  datum <= '2017-01-31 23:59:00' AND sensor=1"
df = pd.DataFrame(list(session.execute(query)))

Clean the dataframe

In [49]:
df['temperatureat2meter'] = df['temperatureat2meter'].replace(to_replace=[-888.8, -777.7, -999.9], value=np.NaN)

In [50]:
df.isnull().values.any()

False

In [51]:
df.tail()

,sensor,datum,temperatureat2meter
3842,1,2017-01-31 23:00:01,0.0
3843,1,2017-01-31 23:10:01,0.1
3844,1,2017-01-31 23:20:01,0.1
3845,1,2017-01-31 23:30:01,0.2
3846,1,2017-01-31 23:50:11,0.0


In [52]:
df_train = pd.DataFrame(df.iloc[:1500])
df_test = pd.DataFrame(df.iloc[1501:])

In [53]:
df_train.tail()

,sensor,datum,temperatureat2meter
1495,1,2017-01-13 01:20:01,-3.2
1496,1,2017-01-13 01:30:01,-3.1
1497,1,2017-01-13 01:40:01,-3.2
1498,1,2017-01-13 02:00:11,-1.7
1499,1,2017-01-13 02:10:01,-1.6


In [54]:
x = np.arange(1500)

In [55]:
x

array([   0,    1,    2, ..., 1497, 1498, 1499])

In [56]:
lin_reg_model = sklearn.linear_model.LinearRegression()

In [61]:
x = x.reshape(-1,1)
y = np.array(df_train.temperatureat2meter)
y = y.reshape(-1,1)

In [62]:
lin_reg_model.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [63]:
x_new = [[1600]]
print(lin_reg_model.predict(x_new))

[[-10.04657056]]
